<a href="https://colab.research.google.com/github/matsunagalab/tutorial_analyzingMDdata/blob/main/01_mdtraj_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MDTraj basics

## MDTrajのインストール

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...


In [ ]:
!conda install -c conda-forge mdtraj

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.5.2

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.5.2



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - mdtraj


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    astunparse-1.6.3           |     pyhd8ed1ab_0          15 KB  conda-forge
    blosc-1.21.4               |       h0f2a231_0          48 KB  conda-forge
    c-blosc2-2.10.0            |       hb4ffafa_0         298 KB  conda-forge
    ca-certificates-2023.5.7 

## T4 LysozymeのMDデータの取得

`wildtype.psf` と `wildtype.dcd` をダウンロードする。

In [ ]:
!gdown 1GeXp7pbLO8nuK5L6x1xzxj1AX6kMnXy6
!gdown 1G_dnmdjgKFr-O8ciNo5JVR9iN4FQBAAE

Downloading...
From: https://drive.google.com/uc?id=1GeXp7pbLO8nuK5L6x1xzxj1AX6kMnXy6
To: /content/wildtype.psf
100% 623k/623k [00:00<00:00, 142MB/s]
Downloading...
From: https://drive.google.com/uc?id=1G_dnmdjgKFr-O8ciNo5JVR9iN4FQBAAE
To: /content/wildtype.dcd
100% 215M/215M [00:01<00:00, 153MB/s]


## MDTrajを使ってみる

以下の文書は MDTrajのドキュメント https://www.mdtraj.org/1.9.7/examples/introduction.html を主に日本語訳したものです。

ディスクからトラジェクトリをロードして始めます。MDTrajは自動的にファイル拡張子を解析し、適切なローダーを使用します。
Topology情報を含まないファイルをロードするには、Gromacs XTCファイルのようなもの、
Topologyを記述する何か、たとえばPDBファイル、をtopキーワード引数で提供する必要があります。

In [ ]:
import mdtraj as md
import numpy as np

# DCDファイルからトラジェクトリを読み込み、対応するPSFトポロジーファイルを使用する
# トラジェクトリオブジェクト t が作成される
t = md.load('/content/wildtype.dcd', top='/content/wildtype.psf')

# t の中身を出力する
print(t)

<mdtraj.Trajectory with 6768 frames, 2643 atoms, 164 residues, and unitcells>


Trajectoryのフレームの一部だけに興味がある場合は、それをスライスすることができます。

In [ ]:
# 最初の10フレームを見てみましょう
print(t[1:10])

<mdtraj.Trajectory with 9 frames, 2643 atoms, 164 residues, and unitcells>


In [ ]:
# それとも最後のフレームを見る？
print(t[-1])

<mdtraj.Trajectory with 1 frames, 2643 atoms, 164 residues, and unitcells>


Trajectoryオブジェクトには多くの情報があります。自明なのは
デカルト座標です。それらはxyzの下のnumpy配列として格納されています。全ての
座標の長さはナノメートルで保存されています。時間単位
はピコ秒です。角度はdegree (radianではない) で保存されています。

Trajectoryオブジェクトが持っているプロパティやメソッドは[ドキュメント](https://www.mdtraj.org/1.9.7/api/generated/mdtraj.Trajectory.html#)で確認することができます。

In [ ]:
t.xyz.shape

(6768, 2643, 3)

In [ ]:
np.mean(t.xyz)

0.37758622

In [ ]:
# 最初の10フレームのシミュレーション時間（ピコ秒）
t.time[0:10]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
# 最後のフレームの単位セルの長さは？（ナノメートル）
t.unitcell_lengths[-1]

array([7.5977883, 7.5977883, 7.5977883], dtype=float32)

Trajectoryをディスクに戻すのは簡単です。

In [ ]:
# hdf5形式はファイル内にトポロジを保存するので便利です
t[::2].save('halftraj.h5')

In [ ]:
# フォーマットは拡張子に基づいて解析されます、またはフォーマット固有のsaveメソッドを呼び出すこともできます
t[0:10].save_dcd('first-ten-frames.dcd')

TrajectoryオブジェクトにはTopologyオブジェクトへが含まれており、便利に利用することが出来ます。例えば、alpha carbon のみを含むトラジェクトリのコピーを保存したい場合、以下のようにできます。

In [ ]:
atoms_to_keep = [a.index for a in t.topology.atoms if a.name == 'CA']
t.restrict_atoms(atoms_to_keep) # この操作はトラジェクトリオブジェクトを変化させます
t.save('CA-only.h5')

Topogyオブジェクトが持つプロパティやメソッドは[ドキュメント](https://www.mdtraj.org/1.9.7/api/generated/mdtraj.Topology.html)で確認することができます。